In [126]:
import pandas as pd
import logging 
import time
from datetime import datetime


# Configure logging to write only to file
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    filename='log.txt',
    filemode='a',
)

print("Logging configured - output will be written to log.txt")

Logging configured - output will be written to log.txt


In [127]:
logging.info("="*80)
logging.info("Starting data loading process...")
start_time = time.time()

try:
    data = {
        'materials': pd.read_excel('materials.xlsx', sheet_name='materials'),
        'plants': pd.read_excel('plants.xlsx', sheet_name='plants'),
        'storage': pd.read_excel('storage.xlsx', sheet_name='storage'),
        'suppliers': pd.read_excel('suppliers.xlsx', sheet_name='suppliers'),
        'supplier_names': pd.read_excel('supplier-names.xlsx', sheet_name='supplier-names'),
        'manufacturer_names': pd.read_excel('manufacturer-names.xlsx', sheet_name='manufacturer-names')
    }
    
    total_time = time.time() - start_time
    logging.info(f"All files loaded successfully in {total_time:.2f}s")
    logging.info("="*80)
    
except FileNotFoundError as e:
    logging.error(f"File not found: {e}")
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")

C:\Users\pan0005a\AppData\Roaming\Python\Python314\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\pan0005a\AppData\Roaming\Python\Python314\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\pan0005a\AppData\Roaming\Python\Python314\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\pan0005a\AppData\Roaming\Python\Python314\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\pan0005a\Ap

In [128]:
logging.info("="*80)
logging.info("Applying Transformations")
# Replace whitespace with 'ACTIVE' in DeletedStorageLevel
data['storage']['DeletedStorageLevel'].replace(' ', 'ACTIVE', inplace=True)

In [129]:
# checking for null values and unique values columnwise in every dataframe
for key, df in data.items():
    print(f"\n{'='*60}")
    print(f"DataFrame: '{key}'")
    print(f"{'='*60}")
    print(f"Number of rows: {len(df)}")
    
    print("\nNull Values:")
    null_counts = df.isna().sum()
    print(null_counts)
    print(f"Total null values: {null_counts.sum()}")
    
    print("\nUnique Values:")
    unique_counts = df.nunique()
    print(unique_counts)


DataFrame: 'materials'
Number of rows: 51

Null Values:
MaterialReference     0
ManufacturerID        0
ArticleNumber         0
TypeCode             19
ShortText             0
dtype: int64
Total null values: 19

Unique Values:
MaterialReference    51
ManufacturerID       10
ArticleNumber        51
TypeCode             32
ShortText            51
dtype: int64

DataFrame: 'plants'
Number of rows: 134

Null Values:
MaterialReference    0
Plant                0
Disposition          0
ReporderPoint        0
dtype: int64
Total null values: 0

Unique Values:
MaterialReference    51
Plant                 4
Disposition           4
ReporderPoint        72
dtype: int64

DataFrame: 'storage'
Number of rows: 937

Null Values:
MaterialReference      0
Plant                  0
StorageLocation        0
StorageBin             0
DeletedStorageLevel    0
dtype: int64
Total null values: 0

Unique Values:
MaterialReference      49
Plant                   4
StorageLocation         5
StorageBin             2

In [130]:
# handling null values
data['materials']['TypeCode'] = data['materials'].apply(
    lambda row: 'TC00000000' + row['MaterialReference'][-2:] if pd.isna(row['TypeCode']) else row['TypeCode'],
    axis=1
)

print(f"Null TypeCode count after filling: {data['materials']['TypeCode'].isna().sum()}")

Null TypeCode count after filling: 0


In [131]:
# Convert Plant column to string to preserve leading zeros
data['plants']['Plant'] = data['plants']['Plant'].astype(str).str.zfill(4)
data['storage']['Plant'] = data['storage']['Plant'].astype(str).str.zfill(4)

logging.info("Transformations applied successfully.")
logging.info("="*80)

In [132]:
# Cleaning data - removing whitespace from all string columns
logging.info("Starting data cleaning process...")
clean_start = time.time()

for key, df in data.items():
    logging.info(f"Cleaning {key} dataset...")
    # Strip whitespace from all string columns
    for col in df.columns:
        if df[col].dtype == 'object':  # String columns
            df[col] = df[col].astype(str).str.strip()
            # Replace empty strings with NaN
            df[col] = df[col].replace('', pd.NA)
            df[col] = df[col].replace('nan', pd.NA)
    
    data[key] = df
    logging.info(f" Cleaned {key}: {len(df)} rows")

clean_time = time.time() - clean_start
logging.info(f"Data cleaning completed in {clean_time:.2f}s")
print(f"Data cleaning completed in {clean_time:.2f}s")


Data cleaning completed in 0.01s


Aggregate data from all sources into the final result format.
    
    The aggregation strategy:
    1. Start with materials as the base
    2. Add manufacturer names via ManufacturerID
    3. Join with plants data via MaterialReference
    4. Join with suppliers data via MaterialReference
    5. Add supplier names via SupplierID
    6. Join with storage data via MaterialReference and Plant

In [133]:
# Step 1: Merge materials with manufacturer names
logging.info("="*80)
logging.info("Starting data aggregation process...")
aggregation_start = time.time()

logging.info("Step 1: Merging materials with manufacturer names...")
step1_start = time.time()
result = data['materials'].merge(
    data['manufacturer_names'],
    on='ManufacturerID',
    how='left'
)
result.drop(columns=['ManufacturerID'], inplace=True)
logging.info(f"Step 1 completed: {len(result)} rows in {time.time() - step1_start:.2f}s")

result.head(5)

,MaterialReference,ArticleNumber,TypeCode,ShortText,ManufacturerName
0,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.
1,MAT-0000000002,ART0000000002,TC0000000002,Short text 2,Trusted Materials Inc.
2,MAT-0000000003,ART0000000003,TC0000000003,Short text 3,Innovative Materials Ltd.
3,MAT-0000000004,ART0000000004,TC0000000004,Short text 4,Advanced Components
4,MAT-0000000005,ART0000000005,TC0000000005,Short text 5,Reliable Parts Co.


In [134]:
# Step 2: Merge with plants data
logging.info("Step 2: Merging with plants data...")
step2_start = time.time()
result = result.merge(data['plants'], on='MaterialReference', how='left')
logging.info(f"Step 2 completed: {len(result)} rows in {time.time() - step2_start:.2f}s")

result.head(5)

,MaterialReference,ArticleNumber,TypeCode,ShortText,ManufacturerName,Plant,Disposition,ReporderPoint
0,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.,0020,LS,50
1,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.,0030,TW,48
2,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.,0040,PV,70
3,MAT-0000000002,ART0000000002,TC0000000002,Short text 2,Trusted Materials Inc.,0010,LS,30
4,MAT-0000000002,ART0000000002,TC0000000002,Short text 2,Trusted Materials Inc.,0020,PV,50


In [135]:
# Step 3: Merge with suppliers data
logging.info("Step 3: Merging with suppliers data...")
step3_start = time.time()
result = result.merge(data['suppliers'], on=['MaterialReference'], how='left')
logging.info(f"Step 3 completed: {len(result)} rows in {time.time() - step3_start:.2f}s")

result.head(5)

,MaterialReference,ArticleNumber,TypeCode,ShortText,ManufacturerName,Plant,Disposition,ReporderPoint,SupplierID,SupplierArticleNumber
0,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.,0020,LS,50,23456.0,611266177.0
1,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.,0030,TW,48,23456.0,611266177.0
2,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.,0040,PV,70,23456.0,611266177.0
3,MAT-0000000002,ART0000000002,TC0000000002,Short text 2,Trusted Materials Inc.,0010,LS,30,56789.0,848935793.0
4,MAT-0000000002,ART0000000002,TC0000000002,Short text 2,Trusted Materials Inc.,0010,LS,30,23456.0,747936081.0


In [136]:
# Step 4: Add supplier names
logging.info("Step 4: Adding supplier names...")
step4_start = time.time()
result = result.merge(data['supplier_names'], on='SupplierID', how='left')
result.drop(columns=['SupplierID'], inplace=True)
logging.info(f"Step 4 completed: {len(result)} rows in {time.time() - step4_start:.2f}s")

result.head(5)

,MaterialReference,ArticleNumber,TypeCode,ShortText,ManufacturerName,Plant,Disposition,ReporderPoint,SupplierArticleNumber,SupplierName
0,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.,0020,LS,50,611266177.0,Tech Solutions Ltd.
1,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.,0030,TW,48,611266177.0,Tech Solutions Ltd.
2,MAT-0000000001,ART0000000001,TC0000000001,Short text 1,Trusted Materials Inc.,0040,PV,70,611266177.0,Tech Solutions Ltd.
3,MAT-0000000002,ART0000000002,TC0000000002,Short text 2,Trusted Materials Inc.,0010,LS,30,848935793.0,Prime Distributors
4,MAT-0000000002,ART0000000002,TC0000000002,Short text 2,Trusted Materials Inc.,0010,LS,30,747936081.0,Tech Solutions Ltd.


In [137]:
# Step 5: Merge with storage data
logging.info("Step 5: Merging with storage data...")
step5_start = time.time()
result = result.merge(data['storage'], on=['MaterialReference', 'Plant'], how='left')
logging.info(f"Step 5 completed: {len(result)} rows in {time.time() - step5_start:.2f}s")

total_aggregation_time = time.time() - aggregation_start
logging.info(f" Aggregation completed successfully in {total_aggregation_time:.2f}s")
logging.info(f"Final result: {len(result)} rows, {len(result.columns)} columns")
logging.info("="*80)


In [138]:
# Formatting output as per requirement
# Reorder columns: MaterialReference, ManufacturerName, ArticleNumber, TypeCode, ShortText, 
#                  Plant, Disposition, ReporderPoint, SupplierName, SupplierArticleNumber, 
#                  StorageLocation, StorageBin, DeletedStorageLevel

logging.info("Formatting result columns...")
result = result[[
    'MaterialReference',
    'ManufacturerName',
    'ArticleNumber',
    'TypeCode',
    'ShortText',
    'Plant',
    'Disposition',
    'ReporderPoint',
    'SupplierName',
    'SupplierArticleNumber',
    'StorageLocation',
    'StorageBin',
    'DeletedStorageLevel'
]]

logging.info(f"Columns formatted: {list(result.columns)}")
result.head()

,MaterialReference,ManufacturerName,ArticleNumber,TypeCode,ShortText,Plant,Disposition,ReporderPoint,SupplierName,SupplierArticleNumber,StorageLocation,StorageBin,DeletedStorageLevel
0,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0020,LS,50,Tech Solutions Ltd.,611266177.0,SL01,SB001SL01,X
1,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0030,TW,48,Tech Solutions Ltd.,611266177.0,SL01,SB001SL01,ACTIVE
2,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0030,TW,48,Tech Solutions Ltd.,611266177.0,SL01,SB002SL01,X
3,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0030,TW,48,Tech Solutions Ltd.,611266177.0,SL01,SB003SL01,ACTIVE
4,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0030,TW,48,Tech Solutions Ltd.,611266177.0,SL02,SB001SL02,ACTIVE


In [139]:
# Handle null values - fill with N/A
logging.info("Filling null values with 'N/A'...")
null_fill_start = time.time()

# Fill all null values with 'N/A'
result = result.fillna('N/A')

# Verify no null values remain
remaining_nulls = result.isna().sum().sum()
print(f"Remaining null values: {remaining_nulls}")
logging.info(f"Remaining null values after filling: {remaining_nulls}")

result.head()

Remaining null values: 0


,MaterialReference,ManufacturerName,ArticleNumber,TypeCode,ShortText,Plant,Disposition,ReporderPoint,SupplierName,SupplierArticleNumber,StorageLocation,StorageBin,DeletedStorageLevel
0,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0020,LS,50,Tech Solutions Ltd.,611266177.0,SL01,SB001SL01,X
1,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0030,TW,48,Tech Solutions Ltd.,611266177.0,SL01,SB001SL01,ACTIVE
2,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0030,TW,48,Tech Solutions Ltd.,611266177.0,SL01,SB002SL01,X
3,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0030,TW,48,Tech Solutions Ltd.,611266177.0,SL01,SB003SL01,ACTIVE
4,MAT-0000000001,Trusted Materials Inc.,ART0000000001,TC0000000001,Short text 1,0030,TW,48,Tech Solutions Ltd.,611266177.0,SL02,SB001SL02,ACTIVE


In [140]:
# Counting number of 'N/A' values in each column
na_counts = (result == 'N/A').sum()
print("\nN/A counts per column:")
print(na_counts)
print(f"\nTotal N/A values: {na_counts.sum()}")

# counting number of rows which have 'N/A' in any column
na_row_counts = result.isin(['N/A']).any(axis=1).sum()
print(f"\nTotal rows with at least one N/A value: {na_row_counts}")


N/A counts per column:
MaterialReference         0
ManufacturerName          0
ArticleNumber             0
TypeCode                  0
ShortText                 0
Plant                     0
Disposition               0
ReporderPoint             0
SupplierName             79
SupplierArticleNumber    79
StorageLocation          57
StorageBin               57
DeletedStorageLevel      57
dtype: int64

Total N/A values: 329

Total rows with at least one N/A value: 135


In [141]:
# Export result to Excel using openpyxl engine
logging.info("Starting export to Excel...")
export_start = time.time()

try:
    with pd.ExcelWriter('result.xlsx', engine='openpyxl') as writer:
        result.to_excel(writer, sheet_name='result-template', index=False)
    
    export_time = time.time() - export_start
    logging.info(f"Successfully exported {len(result)} rows to result.xlsx in {export_time:.2f}s")
    
except Exception as e:
    logging.error(f"Error exporting to Excel: {e}")